In [553]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [554]:
using SparseArrays
struct Transformation
    T::SparseMatrixCSC{Float64}
    non::Int64
    dim::Int64
end

import Base.transpose
function transpose(A::Transformation)
    return Transformation(transpose(A.T), A.non, A.dim)
end

import Base.adjoint
function adjoint(A::Transformation)
    return Transformation(adjoint(A.T), A.non, A.dim)
end

import Base.*
function *(A::Transformation, B)
    n = size(B, 1)
    m = size(B, 2)
    non = A.non
    dim = A.dim
    if dim * non == n
        if m == n
            return dropzeros!(A.T * B)
        else
            return A.T * B
        end
    elseif 9non == n
        C = zeros(3non, 3)
        E = zeros(n, m)
        T = []
        I = []
        J = []
        V = []
        T1 = zeros(9)
        I0 = [1, 2, 3, 1, 2, 3, 1, 2, 3]
        J0 = [1, 1, 1, 2, 2, 2, 3, 3, 3]
        if dim == 2
            for i in 1:non
                T1 = [A.T[2i-1, 2i-1], A.T[2i, 2i-1], 0, A.T[2i-1, 2i], A.T[2i, 2i], 0, 0, 0, 1]
                Idx = I0 .+ (3i - 3)
                Jdx = J0 .+ (3i - 3)
                append!(I, Idx)
                append!(J, Jdx)
                append!(V, T1)
            end
            fn(x, y) = y
            T = sparse(I, J, V, 3non, 3non, fn)
        else
            T = A.T
        end
        for k in 1:m
            for i in 1:non
                for j = 1:3
                    C[3i-2:3i, j] = B[9i-9+3j-2:9i-9+3j, k]
                end
            end
            D = T * C
            for i in 1:non
                for j = 1:3
                    E[9i-9+3j-2:9i-9+3j, k] = D[3i-2:3i, j]
                end
            end
        end
        return E
    else
        error("*(A::Transformation, B): size missmatch dim * non = $dim * $non ≠ $n.")
    end
end

function *(B, A::Transformation)
    n = size(B, 1)
    m = size(B, 2)
    non = A.non
    dim = A.dim
    if dim * non == n
        if m == n
            return dropzeros!(A.T * B)
        else
            return A.T * B
        end
    elseif 9non == n
        C = zeros(3, 3non)
        E = zeros(n, m)
        T = []
        I = []
        J = []
        V = []
        T1 = zeros(9)
        I0 = [1, 2, 3, 1, 2, 3, 1, 2, 3]
        J0 = [1, 1, 1, 2, 2, 2, 3, 3, 3]
        if dim == 2
            for i in 1:non
                T1 = [A.T[2i-1, 2i-1], A.T[2i, 2i-1], 0, A.T[2i-1, 2i], A.T[2i, 2i], 0, 0, 0, 1]
                Idx = I0 .+ (3i - 3)
                Jdx = J0 .+ (3i - 3)
                append!(I, Idx)
                append!(J, Jdx)
                append!(V, T1)
            end
            fn(x, y) = y
            T = sparse(I, J, V, 3non, 3non, fn)
        else
            T = A.T
        end
        for k in 1:m
            for i in 1:non
                for j = 1:3
                    C[1:3, 3i-3+j] = B[9i-9+3j-2:9i-9+3j, k]
                end
            end
            D = C * T
            for i in 1:non
                for j = 1:3
                    E[9i-9+3j-2:9i-9+3j, k] = D[1:3, 3i-3+j]
                end
            end
        end
        return E
    else
        error("*(B, A::Transformation): size missmatch dim * non = $dim * $non ≠ $n.")
    end
end

* (generic function with 283 methods)

In [555]:
gmsh.open("cylinder.geo")

Info    : Reading 'cylinder.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.00046127s, CPU 0.000463s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0453573s, CPU 0.036617s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.0158661s, CPU 0.006439s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0298934s, CPU 0.015336s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.08088 

In [556]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 1443)


LowLevelFEM.Problem("cylinder", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 8030)

In [557]:
csx(x, y, z) = x
csy(x, y, z) = y
csyy(x, y, z) = -y
cs = FEM.CoordinateSystem([csx, csy, 0], [csyy, csx, 0])
T = FEM.rotateNodes(problem, "body", cs);

In [558]:
Q = Transformation(T, size(T, 1) ÷ 3, 3)

Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  24085, 24086, 24085, 24086, 24087, 24088, 24089, 24088, 24089, 24090], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  24085, 24085, 24086, 24086, 24087, 24088, 24088, 24089, 24089, 24090], [1.0, -2.4492935982947064e-16, 2.4492935982947064e-16, 1.0, 1.0, 1.0, -2.4492935982947064e-16, 2.4492935982947064e-16, 1.0, 1.0  …  0.7712240794707432, 0.6365637589782382, -0.6365637589782382, 0.7712240794707433, 1.0, -0.3976860309299219, -0.917521564216997, 0.917521564216997, -0.3976860309299219, 0.9999999999999998], 24090, 24090), 8030, 3)

In [559]:
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)

("left", 0, 0, 0)

In [560]:
fy(x, y, z) = x
load = FEM.load("right", fy=10, fz=0)

("right", 0, 10, 0)

In [561]:
K = FEM.stiffnessMatrix(problem)
K = Q.T' * K * Q.T
f = FEM.loadVector(problem, [load])
FEM.applyBoundaryConditions!(problem, K, f, [supp])
q = FEM.solveDisplacement(K, f)
q = Q.T * q

24090-element Vector{Float64}:
 -0.00019184200754141772
  0.016861841750326966
  2.8180788583151335e-5
  0.0
  0.0
  0.0
 -0.008208273428356299
  0.01488471717627566
  5.226218364646502e-5
 -0.014392487757367508
  ⋮
 -0.012229909308815446
 -0.004352188310824554
  3.683837686760975e-5
 -0.00023532678390269168
  0.00028311900262628723
  2.0447725036132245e-6
  0.0033378887958016206
 -0.0015289026347458444
 -2.2364126589822576e-5

In [562]:
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [563]:
S2 = Q' * S * Q;

In [564]:
u = FEM.showDoFResults(problem, q, :uvec)
ux = FEM.showDoFResults(problem, q, :ux)
uy = FEM.showDoFResults(problem, q, :uy)
uz = FEM.showDoFResults(problem, q, :uz)
sx = FEM.showDoFResults(problem, S, :sx)
sy = FEM.showDoFResults(problem, S, :sy)
sz = FEM.showDoFResults(problem, S, :sz)
sxy = FEM.showDoFResults(problem, S, :sxy)
syz = FEM.showDoFResults(problem, S, :syz, visible=false)
szx = FEM.showDoFResults(problem, S, :szx)

9

In [565]:
s2x = FEM.showDoFResults(problem, S2, :sx)
s2y = FEM.showDoFResults(problem, S2, :sy)
s2z = FEM.showDoFResults(problem, S2, :sz)
s2xy = FEM.showDoFResults(problem, S2, :sxy)
s2yz = FEM.showDoFResults(problem, S2, :syz, visible=true)
s2zx = FEM.showDoFResults(problem, S2, :szx)

15

In [566]:
p1 = FEM.plotOnPath(problem, "path1", s2yz)
p2 = FEM.plotOnPath(problem, "path2", s2yz)

17

In [567]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [568]:
gmsh.finalize()